In [1]:
import torch
import csv
import re
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [2]:
glove_vectors_file = open('../data/subreddit_glove.csv')
glove_vectors = defaultdict(list)

print(glove_vectors_file.readline())
for line in glove_vectors_file:
    line = line.rstrip().split()
#     print(line)
    vec = []
    for i in range(1, len(line)):
        vec.append(float(line[i]))
    if(len(vec) == 0):
        print(line[0])
    glove_vectors[line[0].rstrip().lstrip()] = vec
print(len(glove_vectors))

subreddit, glove_embedding

54074


In [3]:
# print(list(glove_vectors.items())[:10])
# print(glove_vectors[])

In [4]:
def datum(w1, w2, y):
    try:
#         print(glove_vectors[w1])
        if(y == '1'):
            y_ = np.array([1, 0])
        else:
            y_ = np.array([0, 1])
        return glove_vectors[w1] + glove_vectors[w2], y_
    except(KeyError):
        print(w1, w2)
        return [], float(y)

In [5]:
print(list(glove_vectors)[:10])

['iama', 'askreddit', 'pics', 'funny', 'videos', 'the_donald', 'todayilearned', 'gaming', 'music', 'gifs']


In [6]:
input_file = open('../data/subreddit_subreddit_sentiment.csv')
header = input_file.readline()
updated_dataset = []
X = []
y = []
count = 0
nodes_set = set()
reader = csv.reader(input_file)
for row in reader:
    xi, yi = datum(row[0], row[1], row[2])
    nodes_set.add(row[0])
    nodes_set.add(row[1])
    if(len(xi)== 50):
        print(row[0], row[1])
        continue
    updated_dataset.append((row[0], row[1], row[2]))
    X.append(xi)
    y.append(yi)
    count+=1
input_file.close()
# print(X[:10], y[:10])
print(count)

chinavisa shanghai
984519685419685321 discussthetpp
984519685419685321 badeconomics
984519685419685321 economics
984519685419685321 politicaldiscussion
984519685419685321 bestof
984519685419685321 compsci
chinavisa chinalife
007 daniel_craig
984519685419685321 headphones
solving_515654561114 515654561114
wsgy 292277026596
ccj2 chinavisa
ccj2 chinavisa
primeday july12
amazondailydealsplus july12
primedaydeals july12
primedaydeals july12
primeday july12
7719836421125 rickandmorty
ccj2 chinavisa
24811812513198111524 politics
ccj2 chinavisa
ccj2 chinavisa
571903


In [7]:
print(len(X))
print(len(y))
print(len(list(nodes_set)))

571903
571903
54075


In [8]:
print(X[:2])

[[-1.067171, -0.538129, 0.092765, -1.700053, -0.211605, -0.553921, -0.210141, -1.324045, -0.002701, 0.683839, -0.392884, 0.199599, -0.520829, 0.121875, 0.366962, -0.820817, 0.822135, -0.710301, 0.034493, -0.724932, -1.253203, -0.742047, 0.993264, 0.979976, 0.997122, -0.158813, 0.684043, -0.185424, 0.188149, -1.193722, 0.303661, 1.00182, 1.501472, 0.096321, -1.488822, 0.518621, 0.162796, 1.009731, -0.764225, -0.189533, 0.882099, 0.352815, 2.003869, -0.159903, -0.424588, -0.223358, -0.652405, 1.464438, -0.604766, 1.015925, -0.27438, -0.216422, 0.099514, -0.493893, -0.118693, -0.311265, -0.234571, -0.427715, -0.042948, 0.438444, 0.016811, 0.055943, 0.090314, -0.076771, 0.073945, -0.20134, 0.555296, -0.072008, 0.054354, -0.300587, -0.285106, -0.47273, 0.655343, 0.466034, 0.430163, -0.032024, 0.384274, 0.158283, 0.198633, -0.379633, 0.00762, 0.583329, 0.474871, 0.286514, -0.418746, 0.191644, 0.077984, 0.526642, -0.180296, 0.124801, 0.259186, 0.045849, 0.964448, -0.27426, -0.377845, -0.18586

In [9]:
device = torch.device('cpu')

In [10]:
N, dimen_in, dimen_hidden, dimen_out, dimen_class = 64, 100, 50, 10, 2

In [11]:
# x = torch.randn(N, dimen_in, device=device)
# y = torch.randn(N, dimen_class, device=device)

In [12]:
class RedditDataset(Dataset):
    def __init__(self, X, y, transform=None):
        assert(len(X) == len(y))
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return {'X': np.array(self.X[idx], dtype=float), 'y':np.array(self.y[idx], dtype=float)}

In [13]:
dataset = RedditDataset(X, y)

In [14]:
print(dataset[0])

{'X': array([-1.067171, -0.538129,  0.092765, -1.700053, -0.211605, -0.553921,
       -0.210141, -1.324045, -0.002701,  0.683839, -0.392884,  0.199599,
       -0.520829,  0.121875,  0.366962, -0.820817,  0.822135, -0.710301,
        0.034493, -0.724932, -1.253203, -0.742047,  0.993264,  0.979976,
        0.997122, -0.158813,  0.684043, -0.185424,  0.188149, -1.193722,
        0.303661,  1.00182 ,  1.501472,  0.096321, -1.488822,  0.518621,
        0.162796,  1.009731, -0.764225, -0.189533,  0.882099,  0.352815,
        2.003869, -0.159903, -0.424588, -0.223358, -0.652405,  1.464438,
       -0.604766,  1.015925, -0.27438 , -0.216422,  0.099514, -0.493893,
       -0.118693, -0.311265, -0.234571, -0.427715, -0.042948,  0.438444,
        0.016811,  0.055943,  0.090314, -0.076771,  0.073945, -0.20134 ,
        0.555296, -0.072008,  0.054354, -0.300587, -0.285106, -0.47273 ,
        0.655343,  0.466034,  0.430163, -0.032024,  0.384274,  0.158283,
        0.198633, -0.379633,  0.00762 ,  0.58

In [15]:
dataloader = DataLoader(dataset, batch_size=64,
                        shuffle=True, num_workers=4)

In [16]:
# for i_batch, sample_batched in enumerate(dataloader):
#     print(sample_batched['X'],
#           sample_batched['y'])

In [17]:
model = torch.nn.Sequential(
            torch.nn.Linear(dimen_in, dimen_hidden),
            torch.nn.ReLU(),
            torch.nn.Linear(dimen_hidden, dimen_out),
            torch.nn.ReLU(),
            torch.nn.Linear(dimen_out, dimen_class),
            torch.nn.Sigmoid(),
        ).to(device)

In [18]:
loss_function = torch.nn.MSELoss(reduction='sum')

In [19]:
learning_rate = 1e-5

In [20]:
for epoch in range(5):
    for i_batch, sample_batched in enumerate(dataloader):
        X = sample_batched['X']
        y_ = sample_batched['y']
    #         print(y_)
    #         print(X)
        X = X.float()
        y_ = y_.float()
        y_pred = model(X)
    #         print(y_pred.shape)
        loss = loss_function(y_pred, y_)
        print(i_batch, loss.item())
        model.zero_grad()
        loss.backward()
        with torch.no_grad():
            for param in model.parameters():
                param.data -= learning_rate * param.grad

0 30.906490325927734
1 30.834243774414062
2 30.20039176940918
3 30.600601196289062
4 30.435396194458008
5 31.091398239135742
6 30.75153350830078
7 30.487253189086914
8 30.415876388549805
9 30.6484375
10 30.636686325073242
11 30.443449020385742
12 30.719839096069336
13 30.610580444335938
14 30.393972396850586
15 30.577756881713867
16 30.465288162231445
17 30.15180778503418
18 30.444726943969727
19 30.298791885375977
20 30.368825912475586
21 30.253177642822266
22 30.33041000366211
23 30.518272399902344
24 30.78914451599121
25 30.35095977783203
26 30.666793823242188
27 30.28229522705078
28 30.754573822021484
29 30.347497940063477
30 30.121492385864258
31 30.590282440185547
32 30.30182456970215
33 30.418174743652344
34 30.670196533203125
35 30.57862663269043
36 30.90550422668457
37 30.89069175720215
38 30.49166488647461
39 30.643640518188477
40 30.35477638244629
41 30.571453094482422
42 30.583398818969727
43 30.350757598876953
44 30.259490966796875
45 30.62594985961914
46 30.4112606048584


401 29.078109741210938
402 28.77316665649414
403 28.94317626953125
404 29.183652877807617
405 29.229297637939453
406 29.016695022583008
407 29.12764549255371
408 28.9161376953125
409 29.01552391052246
410 29.182170867919922
411 29.417816162109375
412 29.290924072265625
413 29.57935905456543
414 29.056419372558594
415 29.106706619262695
416 29.306961059570312
417 28.980728149414062
418 29.22085189819336
419 29.246164321899414
420 28.88431167602539
421 29.092344284057617
422 29.132091522216797
423 29.407596588134766
424 29.216228485107422
425 29.321447372436523
426 29.467391967773438
427 28.92015838623047
428 28.89003562927246
429 29.153764724731445
430 28.199007034301758
431 29.365232467651367
432 29.020660400390625
433 28.38382911682129
434 28.90822982788086
435 28.92469024658203
436 28.572938919067383
437 29.545671463012695
438 29.219453811645508
439 28.48982810974121
440 28.36737823486328
441 29.530906677246094
442 29.268329620361328
443 29.661373138427734
444 28.956954956054688
445 

871 27.188444137573242
872 27.502609252929688
873 27.37982177734375
874 27.806642532348633
875 27.506010055541992
876 28.156076431274414
877 28.073461532592773
878 28.059003829956055
879 27.998703002929688
880 27.952762603759766
881 27.866077423095703
882 27.16185188293457
883 27.473758697509766
884 27.547672271728516
885 28.518468856811523
886 27.61555290222168
887 28.190532684326172
888 27.66591453552246
889 27.603158950805664
890 27.59354591369629
891 27.603994369506836
892 27.217079162597656
893 27.767763137817383
894 27.55391502380371
895 26.721893310546875
896 28.458553314208984
897 28.306907653808594
898 28.31039810180664
899 27.902673721313477
900 28.242948532104492
901 28.22629165649414
902 27.84944725036621
903 27.23218536376953
904 27.871490478515625
905 28.147113800048828
906 27.211790084838867
907 28.026817321777344
908 27.69130516052246
909 28.002092361450195
910 27.400279998779297
911 27.15854835510254
912 27.092493057250977
913 27.90433692932129
914 28.32237434387207
91

1319 25.24533462524414
1320 27.177820205688477
1321 25.657169342041016
1322 26.767839431762695
1323 25.233617782592773
1324 25.542659759521484
1325 27.586708068847656
1326 27.198415756225586
1327 26.253528594970703
1328 26.203956604003906
1329 25.642234802246094
1330 26.241003036499023
1331 25.37641143798828
1332 26.027320861816406
1333 27.218748092651367
1334 26.260053634643555
1335 26.522571563720703
1336 25.830795288085938
1337 25.631128311157227
1338 26.722280502319336
1339 25.77883529663086
1340 26.83794593811035
1341 27.40081214904785
1342 26.389245986938477
1343 26.680606842041016
1344 25.934871673583984
1345 27.081188201904297
1346 26.474748611450195
1347 26.007068634033203
1348 26.5052433013916
1349 27.40100860595703
1350 26.592548370361328
1351 26.7023868560791
1352 26.981706619262695
1353 26.455890655517578
1354 28.237125396728516
1355 26.164390563964844
1356 25.710391998291016
1357 25.474655151367188
1358 25.941635131835938
1359 26.548418045043945
1360 25.493000030517578
13

1669 25.763336181640625
1670 25.08791732788086
1671 25.404062271118164
1672 25.10399055480957
1673 24.682170867919922
1674 25.407175064086914
1675 25.69459342956543
1676 25.18113136291504
1677 25.88423728942871
1678 25.701438903808594
1679 25.487401962280273
1680 26.070960998535156
1681 25.04410743713379
1682 26.535152435302734
1683 26.262819290161133
1684 25.825000762939453
1685 24.649383544921875
1686 25.8663330078125
1687 25.537109375
1688 26.559553146362305
1689 26.484487533569336
1690 25.591825485229492
1691 25.659334182739258
1692 25.0433349609375
1693 26.632030487060547
1694 25.01209259033203
1695 26.134593963623047
1696 25.2863712310791
1697 26.56876564025879
1698 25.557235717773438
1699 25.010093688964844
1700 25.631956100463867
1701 25.797483444213867
1702 25.44437026977539
1703 25.90337562561035
1704 25.55707359313965
1705 25.469404220581055
1706 25.71863555908203
1707 25.176429748535156
1708 26.419998168945312
1709 25.36112403869629
1710 26.102148056030273
1711 26.147293090

...

7414 12.760395050048828
7415 8.709464073181152
7416 11.43027400970459
7417 11.442028045654297
7418 8.708617210388184
7419 18.260496139526367
7420 12.807426452636719
7421 10.079327583312988
7422 14.187707901000977
7423 10.062771797180176
7424 10.084196090698242
7425 14.185038566589355
7426 16.907764434814453
7427 14.15025806427002
7428 14.222742080688477
7429 8.710675239562988
7430 15.526101112365723
7431 12.811814308166504
7432 18.32343101501465
7433 11.398733139038086
7434 19.657360076904297
7435 12.775715827941895
7436 14.125670433044434
7437 10.048925399780273
7438 5.975515365600586
7439 12.757333755493164
7440 16.869157791137695
7441 16.885324478149414
7442 14.101163864135742
7443 15.492595672607422
7444 10.089851379394531
7445 14.14571475982666
7446 18.223482131958008
7447 8.731973648071289
7448 11.440652847290039
7449 18.23649024963379
7450 8.766043663024902
7451 14.133552551269531
7452 7.277994155883789
7453 12.807619094848633
7454 7.33389139175415
7455 15.579708099365234
7456 1

7776 10.04892635345459
7777 11.446660041809082
7778 15.5074462890625
7779 11.46639347076416
7780 14.168547630310059
7781 19.616134643554688
7782 14.170207977294922
7783 11.412796974182129
7784 11.457366943359375
7785 15.502647399902344
7786 10.013848304748535
7787 10.061116218566895
7788 8.707990646362305
7789 12.779861450195312
7790 11.490781784057617
7791 10.047638893127441
7792 8.703079223632812
7793 10.023026466369629
7794 14.142964363098145
7795 12.819062232971191
7796 15.539997100830078
7797 16.880081176757812
7798 10.035533905029297
7799 9.980916976928711
7800 12.832982063293457
7801 10.050334930419922
7802 15.552545547485352
7803 16.93385887145996
7804 14.165854454040527
7805 11.43387508392334
7806 14.120534896850586
7807 14.161423683166504
7808 14.142871856689453
7809 10.095874786376953
7810 18.270755767822266
7811 10.059237480163574
7812 14.182841300964355
7813 8.634023666381836
7814 7.317594051361084
7815 12.810752868652344
7816 10.071061134338379
7817 14.180591583251953
781

8146 18.272621154785156
8147 10.052462577819824
8148 11.40685749053955
8149 7.323704242706299
8150 8.668329238891602
8151 12.81705379486084
8152 18.244539260864258
8153 15.51351547241211
8154 11.384130477905273
8155 10.051742553710938
8156 10.037189483642578
8157 11.406333923339844
8158 16.919578552246094
8159 4.588064670562744
8160 11.454928398132324
8161 10.055485725402832
8162 14.157770156860352
8163 16.874534606933594
8164 11.423693656921387
8165 10.077524185180664
8166 16.901094436645508
8167 14.203609466552734
8168 16.874536514282227
8169 9.986830711364746
8170 8.686592102050781
8171 12.789258003234863
8172 11.383522033691406
8173 10.029036521911621
8174 12.713643074035645
8175 14.141007423400879
8176 12.794258117675781
8177 14.117476463317871
8178 11.429100036621094
8179 11.421700477600098
8180 12.736065864562988
8181 14.182482719421387
8182 14.187580108642578
8183 7.304704666137695
8184 14.167524337768555
8185 7.317413330078125
8186 8.677614212036133
8187 15.561766624450684
818

8523 12.79736328125
8524 12.78273868560791
8525 15.44814395904541
8526 15.503241539001465
8527 10.019142150878906
8528 7.284878253936768
8529 12.777698516845703
8530 11.420228004455566
8531 10.092720985412598
8532 15.436023712158203
8533 8.6480131149292
8534 14.102306365966797
8535 15.475677490234375
8536 12.757790565490723
8537 18.21051788330078
8538 18.306190490722656
8539 12.732892036437988
8540 7.341474533081055
8541 8.63757610321045
8542 11.39609146118164
8543 18.265846252441406
8544 14.113523483276367
8545 12.763262748718262
8546 14.042569160461426
8547 19.642797470092773
8548 8.65338134765625
8549 8.63577651977539
8550 19.66847801208496
8551 18.221540451049805
8552 15.481912612915039
8553 5.916097640991211
8554 10.035968780517578
8555 12.742724418640137
8556 11.462807655334473
8557 14.153682708740234
8558 19.61105728149414
8559 18.229936599731445
8560 12.78738784790039
8561 15.50161075592041
8562 10.051980018615723
8563 16.854875564575195
8564 7.313392639160156
8565 9.9861097335

8897 8.646498680114746
8898 11.424172401428223
8899 14.19592571258545
8900 15.498632431030273
8901 10.058316230773926
8902 10.031119346618652
8903 11.421497344970703
8904 15.483651161193848
8905 11.384068489074707
8906 10.082712173461914
8907 19.692384719848633
8908 8.672484397888184
8909 14.135056495666504
8910 14.113890647888184
8911 16.88999366760254
8912 14.134170532226562
8913 11.391331672668457
8914 8.661922454833984
8915 18.249666213989258
8916 7.271640777587891
8917 16.88425636291504
8918 16.856304168701172
8919 12.813515663146973
8920 8.653515815734863
8921 11.401612281799316
8922 11.329797744750977
8923 11.406464576721191
8924 15.537080764770508
8925 10.021928787231445
8926 9.949057579040527
8927 16.885684967041016
8928 14.135104179382324
8929 14.20435905456543
8930 8.672623634338379
8931 15.539274215698242
8932 18.282115936279297
8933 15.474066734313965
8934 18.267295837402344
8935 15.521197319030762


In [21]:
print(type(X))

<class 'torch.Tensor'>


In [23]:
print(model(X))

tensor([[0.8492, 0.1453],
        [0.9109, 0.0885],
        [0.8831, 0.1085],
        [0.8895, 0.0997],
        [0.9252, 0.0738],
        [0.9254, 0.0686],
        [0.9285, 0.0687],
        [0.8846, 0.1218],
        [0.7994, 0.2005],
        [0.9083, 0.0976],
        [0.8286, 0.1638],
        [0.9313, 0.0662],
        [0.9230, 0.0671],
        [0.9010, 0.0994],
        [0.8790, 0.1130],
        [0.9126, 0.0851],
        [0.9285, 0.0749],
        [0.8708, 0.1339],
        [0.9294, 0.0681],
        [0.8843, 0.1178],
        [0.8543, 0.1498],
        [0.8407, 0.1660],
        [0.9499, 0.0447],
        [0.9071, 0.0887],
        [0.8253, 0.1836],
        [0.9350, 0.0593],
        [0.8896, 0.1042],
        [0.9023, 0.0946],
        [0.8231, 0.1664],
        [0.8689, 0.1229],
        [0.8457, 0.1552],
        [0.8870, 0.1083],
        [0.8756, 0.1228],
        [0.8279, 0.1694],
        [0.9432, 0.0539],
        [0.8473, 0.1515],
        [0.8992, 0.0981],
        [0.9056, 0.0984],
        [0.9

In [28]:
dataloader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=4)
feature_file = open('../data/hidden_features.txt', 'w+')
count = 0
for i_batch, sample_batched in enumerate(dataloader):
    output = model[:4](sample_batched['X'].float())
    label = sample_batched['y'].float()
    for i, feature in enumerate(output):
        w1 = updated_dataset[64*i_batch + i][0]
        w2 = updated_dataset[64*i_batch + i][1] 
        feature_file.write(str(w1) + "\t" + str(w2) + "\t")
        for feat_ in output[i].data.numpy():
            feature_file.write(str(feat_)+ "\t")
        feature_file.write("\n")
#         print(w1 , w2, output[i].data.numpy())
#     index = 0
#     for index in range(len(sample_batched)):
#     glove_vectors[updated_dataset[index][0]]
#     print(sample_batched['X'])
#     output = model[:4](sample_batched['X'].float())

In [24]:
print(glove_vectors[updated_dataset[64][0]])

[0.226372, -0.483356, -0.283502, -1.19104, 1.20191, 0.336172, -1.644795, -0.037435, -0.462062, -0.293694, -0.045859, -0.126339, 0.21437, -0.359803, -0.707141, 0.47493, -0.55465, -0.460926, -0.302313, -1.222848, -0.492559, 0.676691, 0.448336, 0.115812, 0.721569, -0.637472, 0.282748, -1.024461, 0.620757, -0.600845, -0.043076, 1.128076, -0.232003, -0.388991, 0.999629, 0.26108, 0.427231, 1.052205, -0.075303, -0.239192, 0.346616, -0.386833, -1.815936, -0.560709, 0.567324, 0.305001, -0.397092, -0.998559, -0.351489, 0.518625]


In [26]:
print(count)

571903
